In [146]:

import xml.etree.ElementTree as et 
path=r'E:\PythonEnv\Trade Analysis\U6787370_multi2_20200320_20210319_AF_NA_1179f69c7a1578dcb20ce38cc802fa21 (1).xml'
xtree = et.parse(path)
xroot = xtree.getroot()

In [33]:
path=r'E:\PythonEnv\Trade Analysis\U6787370_U7662380_20200320_20210319_AF_NA_f2b0a43c9ccb0b7d539fae4f5a9ca2f2.txt'
df=pd.read_csv(path)
relevant_columns=['ClientAccountID','Symbol','UnderlyingSymbol','Strike', 'Expiry', 'Buy/Sell',
       'TradeID', 'Put/Call', 'TradeDate','TransactionType','Quantity','TradePrice','NetCash','FifoPnlRealized','Open/CloseIndicator','ClosePrice']

final_columns=['ClientAccountID','Symbol','Ticker','Strategy','Expiry','Strike','Quantity','TradeDate','ClosePrice']
df=merge_symbol(df)
df=get_strategy(df)

df[final_columns].to_excel('test.xlsx')

In [4]:
import pandas as pd
import numpy as np
# Raw Data
group_columns=['Symbol','Open/CloseIndicator','Buy/Sell','Put/Call']
def merge_symbol(df):
    df['Ticker'] = df['Symbol'].where(
        df['UnderlyingSymbol'].isnull(), df['UnderlyingSymbol'])
    return df


def get_strategy(df):
    df['Strategy'] = None
    df.loc[df['AssetClass'] == 'STK', 'Strategy'] = 'Stock'

    short_put = ((df['Open/CloseIndicator'] == 'O')
                 & (df['Buy/Sell'] == 'SELL')
                 & (df['Put/Call'] == 'P'))
    short_call = ((df['Open/CloseIndicator'] == 'O')
                  & (df['Buy/Sell'] == 'SELL')
                  & (df['Put/Call'] == 'C'))
    long_put = ((df['Open/CloseIndicator'] == 'O')
                & (df['Buy/Sell'] == 'BUY')
                & (df['Put/Call'] == 'P'))
    long_call = ((df['Open/CloseIndicator'] == 'O')
                 & (df['Buy/Sell'] == 'BUY')
                 & (df['Put/Call'] == 'C'))

    df.loc[short_put, 'Strategy'] = 'Short Put'
    df.loc[short_call,'Strategy']='Short Call'
    df.loc[long_put, 'Strategy'] = 'Long Put'
    df.loc[long_call,'Strategy']='Long Call'

    return df

def drop_stock(df):
    df=df[df.Strategy!='Stock']
    return df

def group_trades(df):
    df['Put/Call']=df['Put/Call'].where(df['AssetClass']=='OPT','S')
    df_grouped=df.groupby(group_columns)
    df['TradeDateImp']=group_trade_date(df_grouped)
    df['Quantity']=group_quantity(df_grouped)
    return df

def group_close_price(df):
    df['adjusted_close_volume']=df['Quantity']*df['ClosePrice']
    df['adjusted_close']=df.groupby(group_columns)['adjusted_close_volume'].transform('sum')
    df['adjusted_close']=df['adjusted_close_volume']/df['Quantity']
    
def group_quantity(df_grouped):
    return df_grouped['Quantity'].transform('sum')

def group_trade_date(df_grouped):
    tmp_trade_date_max=df_grouped['TradeDate'].transform('max')
    tmp_trade_date_min=df_grouped['TradeDate'].transform('min')
    return tmp_trade_date_max.where(df['Open/CloseIndicator']=='C',tmp_trade_date_min)

def drop_duplicates(df):
    return df.drop_duplicates(group_columns)

def extract_relevant_columns(df,columns=['ClientAccountID','Symbol','Open/CloseIndicator','Ticker','Strategy','Expiry','Strike','Quantity','TradeDate','ClosePrice']):
    return df[columns]

def merge_open_close(df_rel):
    df_close=df_rel.loc[df_rel['Open/CloseIndicator']=='C'].drop(['Open/CloseIndicator','Ticker','Strategy'],axis=1)
    df_open=df_rel.loc[df_rel['Open/CloseIndicator']=='O'].drop(['Open/CloseIndicator',],axis=1)
    df=df_open.merge(df_close,on=['ClientAccountID','Symbol','Expiry','Strike',],suffixes=('_Open','_Close'),how='left')
    return df

## Analysis
def get_capital_usage(df):
    column='Capital'
    df[column]=None
    df[column]=df[column].where(df.Strategy!='Short Put',(df['Strike']-df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Put',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Short Call',(df['Strike'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Call',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Stock',(df['ClosePrice_Open'])*df['Quantity_Open']*100)
    return df

def get_profit(df):
    column='Profit'
    df[column]=None
    df[column]=df[column].where(df.ClosePrice_Close.isnull(),(df['ClosePrice_Close']-df['ClosePrice_Open'])*df['Quantity_Close']*100)
    return df

def get_trade_status(df):
    column='TradeStatus'
    df[column]=np.where(df['Quantity_Close'].isnull(),'OPEN','CLOSE')
    return df

def get_max_reward(df):
    column='MaxReward'
    df[column]=None
    df[column]=df[column].where(df.Strategy!='Short Put',(df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    df[column]=df[column].where(df.Strategy!='Long Put',(df['Strike']-df['ClosePrice_Open'])*df['Quantity_Open']*100)
    df[column]=df[column].where(df.Strategy!='Short Call',(df['ClosePrice_Open'])*df['Quantity_Open']*-100)
    return df
def get_ROC(df):
    column='ROC'
    df[column]=None
    df[column]=df[column].where(df['Quantity_Close'].isnull(),df['Profit']/df['Capital'])
    return df

def split_date(df_col):
    #df_col=df_col.apply(str)
    df_col=pd.to_datetime(df_col,format='%Y%m%d')
    year = df_col.dt.year
    month = df_col.dt.month
    day = df_col.dt.day
    return year,month,day

def split_df_date(df):
    df['open_year'],df['open_month'],df['open_day']=split_date(df['TradeDate_Open'])
    df['close_year'],df['close_month'],df['close_day']=split_date(df['TradeDate_Close'])
    return df

def get_days_in_trade(df):
    df['DaysInTrade']=df['TradeDate_Close']-df['TradeDate_Open']+1
    return df

def get_profit_per_day(df):
    df['ProfitPerDay']=df['Profit']/df['DaysInTrade']
    return df

def sort_by_status(df):
    return df.sort_values(by=['TradeStatus'])
path=r'E:\PythonEnv\Trade Analysis\U6787370_U7662380_20200320_20210319_AF_NA_f2b0a43c9ccb0b7d539fae4f5a9ca2f2.txt'
df=pd.read_csv(path)
df=merge_symbol(df)
df=get_strategy(df)
df=drop_stock(df)
df=group_trades(df)
df=drop_duplicates(df)
df=extract_relevant_columns(df)
df=merge_open_close(df)
df=get_capital_usage(df)
df=get_trade_status(df)
df=get_profit(df)
df=get_max_reward(df)
df=get_ROC(df)
df=split_df_date(df)
df=get_days_in_trade(df)
df=get_profit_per_day(df)
df=sort_by_status(df)
df.to_excel('test2.xlsx')
df.TradeStatus


0     CLOSE
19    CLOSE
25    CLOSE
53    CLOSE
28    CLOSE
32    CLOSE
33    CLOSE
18    CLOSE
38    CLOSE
40    CLOSE
45    CLOSE
46    CLOSE
49    CLOSE
50    CLOSE
51    CLOSE
39    CLOSE
17    CLOSE
27    CLOSE
15    CLOSE
16    CLOSE
9     CLOSE
10    CLOSE
5     CLOSE
4     CLOSE
8     CLOSE
12    CLOSE
11    CLOSE
13    CLOSE
2     CLOSE
14    CLOSE
6     CLOSE
48     OPEN
3      OPEN
44     OPEN
1      OPEN
52     OPEN
43     OPEN
42     OPEN
41     OPEN
47     OPEN
36     OPEN
37     OPEN
35     OPEN
34     OPEN
31     OPEN
30     OPEN
29     OPEN
26     OPEN
24     OPEN
23     OPEN
22     OPEN
21     OPEN
20     OPEN
7      OPEN
54     OPEN
Name: TradeStatus, dtype: object

In [27]:
import pandas as pd
import gspread

gc = gspread.oauth()
gc.open('My Options Journal')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=501908068778-ktel7qngot2r72ju91ec5huimqnphpuh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65318%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=WmB91oKWfOHeR40zD11jTHiQhRGpsf&access_type=offline


In [5]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

sheet=sheets_login()
sheet.open
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()

AttributeError: 'Resource' object has no attribute 'open'

In [25]:
from googleapiclient.discovery import build

def gsheet_api_check(SCOPES,folderpath='./credentials/'):
    creds = None
    if os.path.exists(f'{folderpath}token.pickle'):
        with open(f'{folderpath}token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                f'{folderpath}credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open(f'{folderpath}token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return result
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '12Cxci_PBE8ZG1-IdUX1h_SbnjKIU4yRcf8lEQ14ILSc'
DATA_TO_PULL = 'data'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df2 =get_as_dataframe(data)
df2

COMPLETE: Data copied


AttributeError: 'dict' object has no attribute 'spreadsheet'

In [15]:
import gspread_dataframe as gd
import gspread
# Connecting with `gspread` here
gc = sheets_login()
ws = gc.open("SheetName").worksheet("xyz")
existing = gd.get_as_dataframe(ws)
updated = existing.append(your_new_data)
gd.set_with_dataframe(ws, updated)

AttributeError: 'Resource' object has no attribute 'open'